In [1]:
import os
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator

2025-10-23 13:25:04.978737: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-10-23 13:25:05.034554: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2025-10-23 13:25:05.261650: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-10-23 13:25:05.261758: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-10-23 13:25:05.303481: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to

In [8]:
BASE_DIR = "Aivine/raw_data/archive/dataset_flat_structure"
TRAIN_DIR = "/home/faisal/code/MrZeroX1/Aivine/raw_data/archive/dataset_flat_structure/train"
VAL_DIR   = "/home/faisal/code/MrZeroX1/Aivine/raw_data/archive/dataset_flat_structure/validation"
TEST_DIR  = "/home/faisal/code/MrZeroX1/Aivine/raw_data/archive/dataset_flat_structure/test"


IMG_SIZE = (224,224)
BATCH_SIZE = 32

In [7]:
import os

BASE_DIR  = os.path.join(os.getcwd(), "raw_data/archive/dataset_flat_structure")
TRAIN_DIR = os.path.join(BASE_DIR, "train")
VAL_DIR   = os.path.join(BASE_DIR, "validation")
TEST_DIR  = os.path.join(BASE_DIR, "test")

print("Train Dir:", TRAIN_DIR)
print("Val Dir:  ", VAL_DIR)
print("Test Dir: ", TEST_DIR)


Train Dir: /home/faisal/code/MrZeroX1/Aivine/raw_data/archive/dataset_flat_structure/train
Val Dir:   /home/faisal/code/MrZeroX1/Aivine/raw_data/archive/dataset_flat_structure/validation
Test Dir:  /home/faisal/code/MrZeroX1/Aivine/raw_data/archive/dataset_flat_structure/test


In [9]:
def get_plant_type(folder_name):
    return folder_name.split("_")[0]

all_folders = [f for f in os.listdir(TRAIN_DIR) if os.path.isdir(os.path.join(TRAIN_DIR,f))]
plant_classes = sorted(list({get_plant_type(f) for f in all_folders}))
num_plant_classes = len(plant_classes)

In [10]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    zoom_range=0.2
)
val_test_datagen = ImageDataGenerator(rescale=1./255)


In [11]:
train_gen = train_datagen.flow_from_directory(
    TRAIN_DIR,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',  # disease classes
    shuffle=True
)
val_gen = val_test_datagen.flow_from_directory(
    VAL_DIR,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False
)
test_gen = val_test_datagen.flow_from_directory(
    TEST_DIR,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False
)

num_disease_classes = len(train_gen.class_indices)
print("Plant classes:", plant_classes)
print("Disease classes:", num_disease_classes)

Found 11500 images belonging to 23 classes.
Found 6149 images belonging to 23 classes.
Found 3076 images belonging to 23 classes.
Plant classes: ['Chrysanthemum', 'Hibiscus', 'Money Plant', 'Rose', 'Turmeric']
Disease classes: 23


In [12]:
def plant_generator(directory, batch_size):
    while True:
        for folder in os.listdir(directory):
            folder_dir = os.path.join(directory, folder)
            if not os.path.isdir(folder_dir):
                continue
            imgs = [os.path.join(folder_dir,f) for f in os.listdir(folder_dir) if f.lower().endswith(('.jpg','.png','.jpeg'))]
            for i in range(0,len(imgs),batch_size):
                batch_imgs, batch_labels = [], []
                for img_path in imgs[i:i+batch_size]:
                    img = tf.keras.utils.load_img(img_path,target_size=IMG_SIZE)
                    img_array = tf.keras.utils.img_to_array(img)/255.0
                    batch_imgs.append(img_array)
                    batch_labels.append(plant_classes.index(get_plant_type(os.path.basename(os.path.dirname(img_path)))))
                if batch_imgs:
                    yield tf.convert_to_tensor(batch_imgs), tf.keras.utils.to_categorical(batch_labels,num_plant_classes)

plant_train_gen = plant_generator(TRAIN_DIR,BATCH_SIZE)
plant_val_gen   = plant_generator(VAL_DIR,BATCH_SIZE)


In [28]:
print("Plant classes:")
for idx, name in enumerate(plant_classes):
    print(f"{idx}: {name}")


Plant classes:
0: Chrysanthemum
1: Hibiscus
2: Money Plant
3: Rose
4: Turmeric


In [29]:
disease_classes = train_gen.class_indices
sorted_diseases = sorted(disease_classes.items(), key=lambda x: x[1])

print("\nDisease classes:")
for name, idx in sorted_diseases:
    print(f"{idx}: {name}")

num_disease_classes = len(disease_classes)
print("\nNumber of disease classes:", num_disease_classes)



Disease classes:
0: Chrysanthemum_Bacterial_Leaf_Spot
1: Chrysanthemum_Healthy
2: Chrysanthemum_Septoria_Leaf_Spot
3: Hibiscus_Blight
4: Hibiscus_Healthy
5: Hibiscus_Necrosis
6: Hibiscus_Scorch
7: Money Plant_Money_Plant_Bacterial_Wilt
8: Money Plant_Money_Plant_Chlorosis
9: Money Plant_Money_Plant_Healthy
10: Money Plant_Money_Plant_Manganese_Toxicity
11: Rose_Black_Spot
12: Rose_Downy_Mildew
13: Rose_Healthy
14: Rose_Mosaic_Virus
15: Rose_Powdery_Mildew
16: Rose_Rust
17: Rose_Yellow_Mosaic_Virus
18: Turmeric_Aphid_Infestation
19: Turmeric_Blotch
20: Turmeric_Healthy
21: Turmeric_Leaf_Necrosis
22: Turmeric_Leaf_Spot

Number of disease classes: 23


In [16]:
def build_mobilenetv3(num_classes):
    base = tf.keras.applications.MobileNetV3Small(input_shape=(224,224,3), include_top=False, weights='imagenet')
    base.trainable = False
    model = models.Sequential([
        base,
        layers.GlobalAveragePooling2D(),
        layers.Dense(128,activation='relu'),
        layers.Dropout(0.3),
        layers.Dense(num_classes,activation='softmax')
    ])
    return model

plant_model = build_mobilenetv3(num_plant_classes)
disease_model = build_mobilenetv3(num_disease_classes)

4334752/4334752 [==============================] - 1s 0us/step


In [17]:
plant_model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])
disease_model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [18]:
from tensorflow.keras.callbacks import EarlyStopping
es = EarlyStopping(patience=5,restore_best_weights=True,monitor='val_loss',verbose=1)


steps_train_plant = sum([len(os.listdir(os.path.join(TRAIN_DIR,f))) for f in os.listdir(TRAIN_DIR)])//BATCH_SIZE
steps_val_plant   = sum([len(os.listdir(os.path.join(VAL_DIR,f)))   for f in os.listdir(VAL_DIR)])  //BATCH_SIZE

In [19]:
steps_train_disease = steps_train_plant
steps_val_disease   = steps_val_plant

In [20]:
plant_history = plant_model.fit(
    plant_train_gen,
    steps_per_epoch=steps_train_plant,
    validation_data=plant_val_gen,
    validation_steps=steps_val_plant,
    epochs=50,
    callbacks=[es]
)

Epoch 1/50
 19/359 [>.............................] - ETA: 5:24 - loss: 1.2900 - accuracy: 0.8104

KeyboardInterrupt: 

In [ ]:
disease_history = disease_model.fit(
    train_gen,
    steps_per_epoch=steps_train_disease,
    validation_data=val_gen,
    validation_steps=steps_val_disease,
    epochs=50,
    callbacks=[es]
)

In [ ]:
res_plant = plant_model.evaluate(plant_val_gen, steps=steps_val_plant, verbose=1)
print(f"\nPlant Type Model Accuracy: {res_plant[1]*100:.2f}%")

In [ ]:
res_disease = disease_model.evaluate(val_gen, steps=steps_val_disease, verbose=1)
print(f"Disease Model Accuracy: {res_disease[1]*100:.2f}%")